<a href="https://colab.research.google.com/github/Krumpu/Homework_ICA/blob/main/HW_2_ICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Preparação Dos Dados


*   ### ETAPA 1
Irei dropar BOD e manter COD para a regressão

Outcome = COD

Preditoers = avg_outflow,	avg_inflow,	total_grid,	Am,	COD,	TN,	T,	TM,	Tm,	SLP,	H,	PP,	VV,	V,	VM,	VG   (15) no total

Número de observações: 1382
*   ### ETAPA 2
Separar o dataset de treino e de teste

Visualizar assimetria, e aplicar tratamento para assimetria em valores com assimetria que não estejam entre -1 1  

Normalizar os dados que estão em escalas muito diferentes do resto


* ### ETAPA 3

aplicar OLS nos dados normalizados
$$\beta = (X^T X)^{-1} X^T y$$

Previsão com base no beta obtido
$\hat{y} = X \cdot \beta$

Calcular RMSE
$\sqrt{\frac{1}{N} \sum (y_{real} - y_{pred})^2}$\
Calcular R^2
 $1 - \frac{\text{Soma dos Erros Quadráticos}}{\text{Soma da Variância Total}}$

* ### Etapa 4 Ridge

$$\beta = (X^T X + \lambda I)^{-1} X^T y$$

In [102]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [103]:
url = 'https://raw.githubusercontent.com/Krumpu/Homework_ICA/main/Data-Melbourne_F.csv'
df = pd.read_csv(url)
df = df.drop(columns=['BOD','month', 'day','year'])
df
df_y = df[['COD']].copy()
df_x = df.drop(columns=['COD'])

In [104]:
df_y

,COD
0,730.0
1,740.0
2,836.0
3,850.0
4,1016.0
...,...
1377,660.0
1378,700.0
1379,860.0
1380,900.0


In [105]:
df_x

,avg_outflow,avg_inflow,total_grid,Am,TN,T,TM,Tm,SLP,H,PP,VV,V,VM,VG
0,2.941,2.589,175856,27.0,60.378,19.3,25.1,12.6,0.0,56,1.52,10.0,26.9,53.5,79.5
1,2.936,2.961,181624,25.0,60.026,17.1,23.6,12.3,0.0,63,0.00,10.0,14.4,27.8,0.0
2,2.928,3.225,202016,42.0,64.522,16.8,27.2,8.8,0.0,47,0.25,10.0,31.9,61.1,77.8
3,2.928,3.354,207547,36.0,63.000,14.6,19.9,11.1,0.0,49,0.00,10.0,27.0,38.9,53.5
4,2.917,3.794,202824,46.0,65.590,13.4,19.1,8.0,0.0,65,0.00,10.0,20.6,35.2,53.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,5.068,4.716,305938,28.0,58.807,6.8,10.0,4.0,0.0,56,0.00,10.0,14.4,22.2,0.0
1378,5.882,5.510,293446,40.0,60.671,7.6,16.0,3.0,0.0,71,0.00,0.0,13.5,25.9,0.0
1379,4.638,4.811,307968,47.0,64.000,9.7,16.0,3.0,0.0,65,0.00,10.0,18.1,25.9,0.0
1380,5.373,4.929,316675,46.0,62.479,10.9,17.0,5.0,0.0,65,0.00,10.0,21.9,35.2,51.9


In [106]:

treino_x, teste_x, treino_y, teste_y = train_test_split(df_x,df_y,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        shuffle=True)

In [107]:
assimetria = df.drop('COD', axis=1).skew()
print("--- Assimetria das Variáveis ---")
print(assimetria.sort_values(ascending=False))

--- Assimetria das Variáveis ---
VV             22.195410
SLP            16.553264
PP              8.763062
avg_inflow      3.751865
V               0.833200
VM              0.552547
T               0.552359
Tm              0.498749
TM              0.485291
avg_outflow     0.380545
Am              0.310626
VG              0.286864
total_grid     -0.148416
TN             -0.256672
H              -0.809744
dtype: float64


Os valores de assimetria que fogem do intervalo de -1 a 1 estão acima de 1 portanto iremos usar logaritmo para diminuir os valores que são muito maiores.

OLS SEM TRATAMENTO DE ASSIMETRIA





In [108]:
media_treino = np.mean(treino_x, axis=0)
std_treino = np.std(treino_x, axis=0)
std_treino[ std_treino == 0 ] = 1.0
treino_x_escalado = (treino_x - media_treino) / std_treino
teste_x_escalado  = (teste_x - media_treino) / std_treino

In [109]:

#Coluna para o bias, somente 1's por enquanto
uns_treino = np.ones((treino_x_escalado.shape[0], 1))
X_treino_bias = np.c_[uns_treino, treino_x_escalado]

#Transposa de X
x_transposto = X_treino_bias.T
#Transposta de X * X
xT_x = x_transposto @ X_treino_bias
#Inversa
xT_x_inv = np.linalg.inv(xT_x)

xT_y = x_transposto @ treino_y

beta = xT_x_inv @ xT_y

In [110]:
teste_bias = np.ones((teste_x_escalado.shape[0],1))
teste_x_bias = np.c_[teste_bias, teste_x_escalado]
predicao_ols_y = teste_x_bias @ beta

In [111]:
predicao_ols_y

,COD
0,699.530297
1,868.372827
2,867.792421
3,880.461444
4,787.597943
...,...
272,797.732687
273,895.548086
274,878.478886
275,881.387300


In [112]:
erros = teste_y.values.reshape(-1, 1) - predicao_ols_y
erros_quadrados = erros ** 2
mse = np.mean(erros_quadrados)
rmse_manual = np.sqrt(mse)
print(f"RMSE Manual antes da : {rmse_manual:.4f}")

RMSE Manual antes da : 93.0395


In [113]:
ss_res = np.sum(erros_quadrados)
media_y_real = np.mean(teste_y)
ss_tot = np.sum((teste_y.values.reshape(-1, 1) - media_y_real) ** 2)
r2_manual = 1 - (ss_res / ss_tot)

print(f"R² Manual: {r2_manual.item():.4f}")

R² Manual: 0.5315


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [114]:
colunas_assimetricas =  ['VV', 'SLP', 'PP', 'avg_inflow']

for colunas in colunas_assimetricas:
  treino_x[colunas] = np.log1p(treino_x[colunas])
  teste_x[colunas] = np.log1p(teste_x[colunas])

In [115]:
assimetria = treino_x.skew()
print("--- Assimetria das Variáveis ---")
print(assimetria.sort_values(ascending=False))

--- Assimetria das Variáveis ---
SLP            16.552872
PP              5.546905
avg_inflow      1.839270
V               0.841922
T               0.542608
VM              0.517327
Tm              0.506219
TM              0.486119
avg_outflow     0.334408
VG              0.279784
Am              0.009186
TN              0.001491
total_grid     -0.127093
H              -0.855612
VV             -0.941220
dtype: float64


OLS APOS O TRATAMENTO DE ASSIMETRIA


In [116]:
media_treino = np.mean(treino_x, axis=0)
std_treino = np.std(treino_x, axis=0)
std_treino[ std_treino == 0 ] = 1.0
treino_x_escalado = (treino_x - media_treino) / std_treino
teste_x_escalado  = (teste_x - media_treino) / std_treino

In [117]:

#Coluna para o bias, somente 1's por enquanto
uns_treino = np.ones((treino_x_escalado.shape[0], 1))
X_treino_bias = np.c_[uns_treino, treino_x_escalado]

#Transposa de X
x_transposto = X_treino_bias.T
#Transposta de X * X
xT_x = x_transposto @ X_treino_bias
#Inversa
xT_x_inv = np.linalg.inv(xT_x)

xT_y = x_transposto @ treino_y

beta = xT_x_inv @ xT_y

In [118]:
teste_bias = np.ones((teste_x_escalado.shape[0],1))
teste_x_bias = np.c_[teste_bias, teste_x_escalado]
predicao_ols_y = teste_x_bias @ beta

In [119]:
predicao_ols_y

,COD
0,704.249923
1,871.491015
2,870.836369
3,884.280653
4,782.067707
...,...
272,800.726149
273,894.137246
274,874.332577
275,868.544011


In [120]:
erros = teste_y.values.reshape(-1, 1) - predicao_ols_y
erros_quadrados = erros ** 2
mse = np.mean(erros_quadrados)
rmse_manual = np.sqrt(mse)
print(f"RMSE Manual: {rmse_manual:.4f}")

RMSE Manual: 92.7958


In [121]:
ss_res = np.sum(erros_quadrados)
media_y_real = np.mean(teste_y)
ss_tot = np.sum((teste_y.values.reshape(-1, 1) - media_y_real) ** 2)
r2_manual = 1 - (ss_res / ss_tot)

print(f"R² Manual: {r2_manual.item():.4f}")

R² Manual: 0.5339


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


$$\beta = (X^T X + \lambda I)^{-1} X^T y$$